In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from mmnl import MMNL
from numba import njit
from sklearn import metrics
from qmc import QMC

In [2]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data/data.npy')

In [3]:
def summarize_data(model): 
    #model should be in method(draws) format
    result = []
    for file in os.listdir('.\\resultaten\\%s'%(model)):
        if file.endswith('%s_dgp_results.p'%(model)) and len(result)<200:
            infile = open(os.path.join('.\\resultaten\\%s'%(model), file), 'rb')
            opt_list = pickle.load(infile)
            [result.append(i) for i in opt_list]
    
#     pickle.dump( result, open( "./resultaten/%s/mcdgp%s_utsdgp_all_results.p" %(model,model), "wb" ) )
    return result      
        

In [4]:
@njit
def softmax(X,beta, obs, brand):
    # brand: 0, 1, 2, or 3
    # grab x corresponding to brand choice, [display,feature,price], for each alternative choice
    x = np.zeros((4,3))
    k = 0
    for i in range(1, 10, 4):
        x[:,k] = np.array([X[obs, i + j] for j in range(4)])
        k+=1
    # x_t = [np.array([X[obs, i + j] for i in range(2, X.shape[1] - 1, 4)]).reshape((1, 3)) for j in
    #      range(4)]
    num = np.exp(x[brand,:] @ beta)
    denom = np.exp(x @ beta).sum(axis = 0)
    # num_t = np.exp(x[brand]@beta)
    # denom_t = np.sum([np.exp(x[j] @beta) for j in range(4)], axis=0)
    # if np.any(np.isnan(num)) or np.any(np.isnan(denom)):
    #     raise ValueError('num: %f or denom: %a is Nan' % (num, denom))

    return num / denom

In [12]:
# @njit
def probs(X,model,theta):
    #performs matrix product to obtain the probability of every row
    #X should be in format [display, feature, price]
    method = model.split('(')[0]
    R = int(model.split('(')[1][:-1])
    if method == 'QMC':
        delta = QMC(300,3,R)
    elif method == 'SMC':
        delta = np.random.standard_normal((300,3,R))
    elif method == 'BQMC':
        delta = QMC(300,3,R)

    
    beta = theta[:3].reshape(-1,1) + delta * theta[3:].reshape(-1,1)
    P = []
    
    for i in range(X.shape[0]):
        if method == 'BQMC':
            kernel_mean, C_inv, det = MMNL.kernel_gauss(beta[int(X[i,0])-1,:,:],np.ones(4), theta,None)
        for j in range(4):
            if method == 'QMC' or method == 'SMC':
                P.append(np.mean(MMNL.softmax(X,beta[int(X[i,0])-1,:,:],i,j)))
            elif method == 'BQMC':
                f = MMNL.softmax(X, beta[int(X[i,0])-1,:,:], i, j)
                mean = kernel_mean.reshape(1,-1) @ C_inv @ f.T
                P.append(float(mean))

        

    return np.array(P)


In [7]:
# all_probs = probs(X,'QMC(20000)',np.array([1.5,  1.,  -1.1,  0.4,  0.1,  0.6]))
# pickle.dump(all_probs,open('all_probs_QMC(20000)','wb'))

In [8]:
all_probs = pickle.load(open('all_probs_QMC(20000)', 'rb'))

In [9]:
all_probs.shape

(11192,)

In [13]:
def get_results(model_data,model,probs_true=None):
    #model data should be in list format containing OptimizeResult types
    theta_true = np.array([1.5,  1.,  -1.1,  0.4,  0.1,  0.6])
    results ={}
    theta_est = np.zeros((6,))
    mape = 0
    D = len(model_data)
    print(D)
    for run in model_data:
        theta_est[:3] += run.x[:3]/D
        t = np.exp(run.x[3:])/D
        theta_est[3:] += t
    t
    P = probs(X,model,theta_true)
    mape_choice = 100*np.mean(np.abs((probs_true - P)/ probs_true))
    rmse_choice = np.sqrt(metrics.mean_squared_error(probs_true,P))
    mape_par = np.mean(np.abs((theta_true - theta_est) / theta_true)) * 100
    rmse_par = np.sqrt(metrics.mean_squared_error(theta_true,theta_est))

    results = { 'method': model,
                'theta': theta_est,
                'mape_choice': mape_choice,
                'rmse_choice': rmse_choice,
                'mape_par': mape_par,
                'rmse_par': rmse_par
                    
            }
    return results

Get true probabilities based on big QMC model

In [14]:
models = ['SMC(250)','SMC(500)','SMC(1000)','SMC(2000)','QMC(25)','QMC(50)','QMC(75)','QMC(100)',"QMC(125)",'BQMC(10)','BQMC(15)','BQMC(20)','BQMC(25)','BQMC(30)']
allres = {}
for m in models:
    m_data = summarize_data(m)
    res = get_results(m_data,m,all_probs)
    print(res)
    allres[res['method']] = res

200
{'method': 'SMC(250)', 'theta': array([ 1.5117817 ,  1.00506381, -1.10560798,  0.34002675,  0.17902321,
        0.60034943]), 'mape_choice': 2.5098492294073758, 'rmse_choice': 0.005253159648380281, 'mape_par': 15.979401296163012, 'rmse_par': 0.04090126484268389}
200
{'method': 'SMC(500)', 'theta': array([ 1.51480633,  1.00700921, -1.10500041,  0.34174011,  0.17654853,
        0.59960052]), 'mape_choice': 1.6267289338508697, 'rmse_choice': 0.003451110703369419, 'mape_par': 15.553778582351347, 'rmse_par': 0.039890290333324266}
200
{'method': 'SMC(1000)', 'theta': array([ 1.51594188,  1.00626154, -1.10489827,  0.35822338,  0.17600392,
        0.60097414]), 'mape_choice': 1.1557788493119145, 'rmse_choice': 0.002411854514898227, 'mape_par': 14.790779221545536, 'rmse_par': 0.03614824654039247}
200
{'method': 'SMC(2000)', 'theta': array([ 1.51710648,  1.00677436, -1.10599942,  0.36123824,  0.16434264,
        0.60118851]), 'mape_choice': 0.8144835558635896, 'rmse_choice': 0.00168388776684

In [29]:
df = pd.DataFrame.from_dict(allres)
df

,SMC(250),SMC(500),SMC(1000),SMC(2000),QMC(25),QMC(50),QMC(75),QMC(100),QMC(125),BQMC(10),BQMC(15),BQMC(20),BQMC(25),BQMC(30)
mape_choice,2.45981,1.6347,1.14065,0.855547,2.1608,1.55173,1.06741,0.688507,0.575586,7.2909,4.94663,3.12837,2.1608,2.33719
mape_par,15.9794,15.5538,14.7908,12.7657,13.1692,14.4483,16.4124,13.7825,11.4913,54.6094,36.7214,49.9327,49.5322,37.6725
method,SMC(250),SMC(500),SMC(1000),SMC(2000),QMC(25),QMC(50),QMC(75),QMC(100),QMC(125),BQMC(10),BQMC(15),BQMC(20),BQMC(25),BQMC(30)
rmse_choice,0.00503514,0.0034347,0.0024189,0.0017719,0.00364563,0.00272578,0.00187727,0.00120827,0.000947892,0.0141558,0.00966366,0.00599263,0.00364563,0.00449037
rmse_par,0.0409013,0.0398903,0.0361482,0.0316712,0.0455058,0.0391499,0.0401477,0.0342092,0.0284587,0.411355,0.267927,0.267957,0.25873,0.229895
theta,"[1.51178169673906, 1.0050638090977702, -1.1056...","[1.5148063329060164, 1.0070092141952791, -1.10...","[1.5159418791393937, 1.006261543813022, -1.104...","[1.5171064780894166, 1.0067743573014045, -1.10...","[1.5121438946113632, 1.0060363660921254, -1.09...","[1.5145184050789082, 1.0046447229220143, -1.10...","[1.5160030358592522, 1.0066038390544645, -1.10...","[1.5165651172936356, 1.0062042712362975, -1.10...","[1.5167757938262643, 1.0060453446990365, -1.10...","[2.14533570532408, 1.376277316604454, -0.96376...","[1.37552835093776, 0.9704884534932602, -1.0042...","[1.4211771893082172, 1.0044144560447152, -1.00...","[1.3790119519039257, 1.0579664935061635, -1.03...","[1.6652317056805828, 1.2538426796522633, -1.01..."


In [24]:
with open('mytable.tex', 'w') as tf:
     tf.write(df.to_latex())

In [21]:
models = ['SMC(250)','SMC(500)','SMC(1000)','SMC(2000)','QMC(25)','QMC(50)','QMC(75)','QMC(100)',"QMC(125)",'BQMC(10)','BQMC(15)','BQMC(20)','BQMC(25)','BQMC(30)']
for model in models:
    for file in os.listdir('.\\resultaten\\%s'%(model)):
        if file.endswith('%s_timgdp_results.p'%(model)):
            old_name = os.path.join('.\\resultaten\\%s'%(model), file)
            new_name = old_name[:-16]+'dgp_results.p'
            os.rename(old_name,new_name)